In [9]:
# ============================================================================
# ENHANCED AI-DRIVEN AGENTIC FORM FILLER
# ============================================================================

import instructor
import anthropic
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Set, Union, Any, Tuple, Literal
import asyncio
import nest_asyncio
import time
import json
import os
import traceback
import logging
from datetime import datetime
from enum import Enum
from playwright.async_api import async_playwright, Page, ElementHandle, Locator
from abc import ABC, abstractmethod
from dataclasses import dataclass, asdict, field as dataclass_field

# Apply nest_asyncio for Jupyter compatibility
nest_asyncio.apply()

# ============================================================================
# ENHANCED LOGGING
# ============================================================================

class DebugLogger:
    """Enhanced debugging with structured output"""
    
    def __init__(self, log_level=logging.INFO):
        self.logger = logging.getLogger("AIFormFiller")
        self.logger.setLevel(log_level)
        
        # Console handler
        console_handler = logging.StreamHandler()
        formatter = logging.Formatter(
            '%(asctime)s | %(levelname)-8s | %(funcName)-20s | %(message)s',
            datefmt='%H:%M:%S'
        )
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
        
        # Session tracking
        self.session_data = {
            "start_time": datetime.now().isoformat(),
            "errors": [],
            "warnings": [],
            "successes": [],
            "ai_insights": [],
            "element_interactions": [],
            "form_understanding": None,
            "final_results": None
        }
    
    def log_ai_insight(self, insight: str, context: Dict = None):
        """Log AI reasoning insights"""
        self.session_data["ai_insights"].append({
            "timestamp": datetime.now().isoformat(),
            "insight": insight,
            "context": context or {}
        })
        self.logger.info(f"🤖 AI Insight: {insight}")
    
    def log_interaction(self, action: str, element: str, result: str, details: Dict = None):
        """Log element interactions"""
        self.session_data["element_interactions"].append({
            "timestamp": datetime.now().isoformat(),
            "action": action,
            "element": element,
            "result": result,
            "details": details or {}
        })
        self.logger.info(f"🎯 {action} on {element}: {result}")
    
    def log_error(self, message: str, exception: Exception = None, context: Dict = None):
        error_data = {
            "timestamp": datetime.now().isoformat(),
            "message": message,
            "exception": str(exception) if exception else None,
            "context": context or {}
        }
        self.session_data["errors"].append(error_data)
        self.logger.error(f"❌ {message}")
        if exception:
            self.logger.error(f"   {type(exception).__name__}: {exception}")
    
    def log_success(self, message: str, context: Dict = None):
        self.session_data["successes"].append({
            "timestamp": datetime.now().isoformat(),
            "message": message,
            "context": context or {}
        })
        self.logger.info(f"✅ {message}")
    
    def get_session_summary(self) -> Dict:
        """Get summary of session"""
        return {
            "duration": (datetime.now() - datetime.fromisoformat(self.session_data["start_time"])).total_seconds(),
            "errors": len(self.session_data["errors"]),
            "successes": len(self.session_data["successes"]),
            "ai_insights": len(self.session_data["ai_insights"]),
            "interactions": len(self.session_data["element_interactions"])
        }

# Global logger
logger = DebugLogger()

# ============================================================================
# AI MODELS FOR STRUCTURED REASONING
# ============================================================================

class ElementType(str, Enum):
    """Types of form elements"""
    TEXT_INPUT = "text_input"
    EMAIL_INPUT = "email_input"
    PASSWORD_INPUT = "password_input"
    TEXTAREA = "textarea"
    SELECT = "select"
    RADIO = "radio"
    CHECKBOX = "checkbox"
    BUTTON = "button"
    FILE_INPUT = "file_input"
    HIDDEN = "hidden"
    CUSTOM = "custom"

class FieldPurpose(str, Enum):
    """Semantic purpose of fields"""
    NAME = "name"
    EMAIL = "email"
    PASSWORD = "password"
    PHONE = "phone"
    ADDRESS = "address"
    DESCRIPTION = "description"
    SELECTION = "selection"
    AGREEMENT = "agreement"
    QUANTITY = "quantity"
    DATE = "date"
    CUSTOM_TEXT = "custom_text"
    METADATA = "metadata"

@dataclass
class ElementSemantics:
    """AI understanding of element semantics"""
    element_type: ElementType
    field_purpose: FieldPurpose
    confidence: float
    reasoning: str
    related_fields: List[str] = dataclass_field(default_factory=list)
    validation_hints: List[str] = dataclass_field(default_factory=list)
    interaction_notes: List[str] = dataclass_field(default_factory=list)

class FormElementAnalysis(BaseModel):
    """AI analysis of a form element"""
    selector: str
    element_type: ElementType
    field_purpose: FieldPurpose
    semantic_name: str = Field(description="Human-readable name for this field")
    confidence: float = Field(ge=0.0, le=1.0)
    reasoning: str = Field(description="Why this element serves this purpose")
    expected_content: str = Field(description="What kind of content this field expects")
    interaction_strategy: str = Field(description="How to interact with this element")
    related_elements: List[str] = Field(default_factory=list)
    special_handling: Optional[str] = Field(default=None, description="Special handling requirements")

class FormStructureUnderstanding(BaseModel):
    """AI understanding of entire form structure"""
    form_purpose: str
    form_type: str
    elements: List[FormElementAnalysis]
    element_groups: List[Dict[str, List[str]]] = Field(description="Grouped related elements")
    interaction_order: List[str] = Field(description="Recommended order to fill fields")
    special_instructions: List[str] = Field(description="Special form-specific instructions")

class FieldMatchingDecision(BaseModel):
    """AI decision on matching fields to elements"""
    field_name: str
    matched_selector: Optional[str]
    confidence: float = Field(ge=0.0, le=1.0)
    reasoning: str
    alternative_selectors: List[str] = Field(default_factory=list)
    transformation_needed: Optional[str] = Field(default=None, description="How to transform the value")

# ============================================================================
# ENHANCED ELEMENT ANALYSIS
# ============================================================================

class AIElementAnalyzer:
    """AI-powered element analysis using Claude"""
    
    def __init__(self, anthropic_client):
        self.client = anthropic_client
        self.instructor_client = instructor.from_anthropic(anthropic_client)
    
    async def analyze_form_elements(self, element_contexts: List[Dict]) -> FormStructureUnderstanding:
        """Deeply analyze form structure with AI"""
        logger.log_ai_insight(f"Analyzing {len(element_contexts)} form elements")
        
        try:
            # Prepare comprehensive element data
            element_data = []
            for ctx in element_contexts[:40]:  # Limit for token management
                element_data.append({
                    "selector": ctx.get("selector", ""),
                    "tag": ctx.get("tag_name", ""),
                    "attributes": ctx.get("attributes", {}),
                    "text": ctx.get("text_content", ""),
                    "labels": ctx.get("labels", []),
                    "parent_context": ctx.get("parent_text", ""),
                    "nearby_elements": ctx.get("nearby_text", []),
                    "position": ctx.get("position", {}),
                    "visibility": ctx.get("is_visible", True),
                    "enabled": ctx.get("is_enabled", True)
                })
            
            # Create focused prompt for MTG card form
            prompt = f"""Analyze this Magic: The Gathering card creation form.

Form Elements:
{json.dumps(element_data, indent=2)}

Identify each element's purpose for creating MTG cards. Look for:
- Card name field
- Mana cost field (might be "cost", "mana", or use symbols)
- Card type selector (Creature, Instant, Sorcery, etc.)
- Subtype/creature type field
- Rules text area (card abilities/effects)
- Power/Toughness fields (for creatures)
- Rarity selector
- Artist credit field
- Set/expansion fields

For each element, determine:
1. Its semantic type and purpose
2. How to interact with it (click, type, select)
3. Special handling needed (disabled elements, custom controls)
4. Related elements that work together

Group related elements and suggest interaction order."""
            
            # Get AI analysis
            response = await asyncio.to_thread(
                self.instructor_client.chat.completions.create,
                model="claude-3-5-sonnet-20241022",
                response_model=FormStructureUnderstanding,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2000
            )
            
            logger.log_ai_insight(f"Form identified as: {response.form_type} - {response.form_purpose}")
            logger.session_data["form_understanding"] = {
                "purpose": response.form_purpose,
                "type": response.form_type,
                "element_count": len(response.elements),
                "special_instructions": response.special_instructions
            }
            
            return response
            
        except Exception as e:
            logger.log_error("Failed to analyze form with AI", e)
            # Return basic analysis
            return FormStructureUnderstanding(
                form_purpose="Unknown form",
                form_type="generic",
                elements=[],
                element_groups=[],
                interaction_order=[],
                special_instructions=["Manual analysis required"]
            )
    
    async def match_fields_to_elements(
        self, 
        form_data: Dict[str, str], 
        form_understanding: FormStructureUnderstanding
    ) -> List[FieldMatchingDecision]:
        """AI-driven field matching with reasoning"""
        logger.log_ai_insight(f"Matching {len(form_data)} fields to form elements")
        
        try:
            # Prepare element summaries
            element_summaries = []
            for elem in form_understanding.elements[:20]:  # Limit to avoid token issues
                element_summaries.append({
                    "selector": elem.selector,
                    "semantic_name": elem.semantic_name,
                    "purpose": elem.field_purpose.value,
                    "expected_content": elem.expected_content,
                    "type": elem.element_type.value
                })
            
            # Create prompt with structured format
            prompt = f"""Match these Magic card data fields to the form elements.

CARD DATA TO FILL:
{json.dumps(form_data, indent=2)}

AVAILABLE FORM ELEMENTS:
{json.dumps(element_summaries, indent=2)}

Return ONLY a JSON array with one object per data field. Each object must have these exact fields:
{{
  "field_name": "name from card data",
  "matched_selector": "selector from form elements or null",
  "confidence": 0.8,
  "reasoning": "why this match makes sense",
  "alternative_selectors": [],
  "transformation_needed": null
}}

Match based on semantic meaning:
- "rules" should match text areas for card text/abilities
- "power"/"toughness" should match stat/number fields
- "rarity" should match rarity selectors
- "cost" should match mana cost fields

Return ONLY the JSON array, no other text."""

            # Get AI response
            response = await asyncio.to_thread(
                self.client.messages.create,
                model="claude-3-5-sonnet-20241022",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2000
            )
            
            # Parse response more carefully
            content = response.content[0].text.strip()
            
            # Try to extract JSON array
            try:
                # First try direct parsing
                decisions_data = json.loads(content)
            except json.JSONDecodeError:
                # Try to find JSON array in the content
                import re
                # Look for array pattern
                array_match = re.search(r'\[\s*\{[\s\S]*\}\s*\]', content, re.MULTILINE)
                if array_match:
                    decisions_data = json.loads(array_match.group())
                else:
                    # Try line by line parsing for multiple JSON objects
                    decisions_data = []
                    for line in content.split('\n'):
                        line = line.strip()
                        if line.startswith('{') and line.endswith('}'):
                            try:
                                obj = json.loads(line)
                                decisions_data.append(obj)
                            except:
                                pass
                    
                    if not decisions_data:
                        raise ValueError("Could not parse JSON from AI response")
            
            # Validate and create decision objects
            decisions = []
            for item in decisions_data:
                try:
                    # Ensure required fields have defaults
                    item.setdefault('alternative_selectors', [])
                    item.setdefault('transformation_needed', None)
                    item.setdefault('confidence', 0.5)
                    
                    decision = FieldMatchingDecision(**item)
                    decisions.append(decision)
                    
                    logger.log_ai_insight(
                        f"Matched '{decision.field_name}' to '{decision.matched_selector}' "
                        f"(confidence: {decision.confidence:.2f})"
                    )
                except Exception as e:
                    logger.log_error(f"Failed to parse decision: {item}", e)
            
            # Ensure we have decisions for all fields
            matched_fields = {d.field_name for d in decisions}
            for field_name in form_data.keys():
                if field_name not in matched_fields:
                    decisions.append(FieldMatchingDecision(
                        field_name=field_name,
                        matched_selector=None,
                        confidence=0.0,
                        reasoning="No match found by AI"
                    ))
            
            return decisions
                
        except Exception as e:
            logger.log_error("Failed to match fields with AI", e)
            # Return fallback matches for all fields
            return [
                FieldMatchingDecision(
                    field_name=field,
                    matched_selector=None,
                    confidence=0.0,
                    reasoning="AI matching failed",
                    alternative_selectors=[],
                    transformation_needed=None
                ) for field in form_data.keys()
            ]

# ============================================================================
# ENHANCED ELEMENT EXTRACTION
# ============================================================================

class SmartElementExtractor:
    """Extract rich element information for AI analysis"""
    
    def __init__(self, page: Page):
        self.page = page
    
    async def extract_element_data(self, element: ElementHandle) -> Dict:
        """Extract comprehensive element data"""
        try:
            # Get all basic properties in parallel
            tag_name, attributes, text, bbox, is_visible, is_enabled = await asyncio.gather(
                self._get_tag_name(element),
                self._get_all_attributes(element),
                element.text_content(),
                element.bounding_box(),
                element.is_visible(),
                element.is_enabled()
            )
            
            # Build selector
            selector = self._build_selector(tag_name, attributes)
            
            # Get context
            labels = await self._find_labels(element, attributes.get('id', ''))
            parent_text = await self._get_parent_text(element)
            nearby_text = await self._get_nearby_text(element)
            
            return {
                "selector": selector,
                "tag_name": tag_name,
                "attributes": attributes,
                "text_content": (text or "").strip()[:100],
                "labels": labels,
                "parent_text": parent_text,
                "nearby_text": nearby_text,
                "position": {
                    "x": bbox['x'] if bbox else 0,
                    "y": bbox['y'] if bbox else 0,
                    "width": bbox['width'] if bbox else 0,
                    "height": bbox['height'] if bbox else 0
                },
                "is_visible": is_visible,
                "is_enabled": is_enabled
            }
            
        except Exception as e:
            logger.log_error(f"Failed to extract element data", e)
            return {}
    
    async def _get_tag_name(self, element: ElementHandle) -> str:
        """Get element tag name"""
        try:
            return await element.evaluate("el => el.tagName.toLowerCase()")
        except:
            return "unknown"
    
    async def _get_all_attributes(self, element: ElementHandle) -> Dict[str, str]:
        """Get all element attributes efficiently"""
        try:
            # Get all attributes in one call
            attributes = await element.evaluate("""
                (el) => {
                    const attrs = {};
                    if (el.attributes) {
                        for (let i = 0; i < el.attributes.length; i++) {
                            const attr = el.attributes[i];
                            attrs[attr.name] = attr.value;
                        }
                    }
                    return attrs;
                }
            """)
            return attributes or {}
        except Exception as e:
            logger.log_error("Failed to get attributes", e)
            return {}
    
    def _build_selector(self, tag_name: str, attributes: Dict[str, str]) -> str:
        """Build reliable selector"""
        if attributes.get('id'):
            return f"#{attributes['id']}"
        elif attributes.get('name'):
            return f"{tag_name}[name='{attributes['name']}']"
        elif attributes.get('data-testid'):
            return f"[data-testid='{attributes['data-testid']}']"
        else:
            # Use multiple attributes for uniqueness
            attr_selectors = []
            for attr, value in attributes.items():
                if attr not in ['style', 'class'] and value:
                    attr_selectors.append(f"[{attr}='{value[:20]}']")
            if attr_selectors:
                return tag_name + ''.join(attr_selectors[:2])
            return tag_name
    
    async def _find_labels(self, element: ElementHandle, element_id: str) -> List[str]:
        """Find associated labels"""
        labels = []
        try:
            # Check for labels with 'for' attribute
            if element_id:
                label_elements = await self.page.query_selector_all(f'label[for="{element_id}"]')
                for label in label_elements:
                    text = await label.text_content()
                    if text:
                        labels.append(text.strip())
            
            # Check if element is wrapped in label
            parent_label = await element.evaluate("""
                el => {
                    let parent = el.parentElement;
                    while (parent && parent.tagName !== 'LABEL') {
                        parent = parent.parentElement;
                    }
                    return parent ? parent.textContent : null;
                }
            """)
            if parent_label:
                labels.append(parent_label.strip())
            
            # Check for aria-label
            aria_label = await element.get_attribute('aria-label')
            if aria_label:
                labels.append(aria_label)
                
        except Exception as e:
            logger.log_error("Failed to find labels", e)
        
        return labels[:3]  # Limit to 3 most relevant
    
    async def _get_parent_text(self, element: ElementHandle) -> str:
        """Get parent element text"""
        try:
            parent_text = await element.evaluate("""
                el => {
                    const parent = el.parentElement;
                    if (!parent) return '';
                    // Get text but exclude this element's text
                    const clone = parent.cloneNode(true);
                    const thisClone = clone.querySelector(el.tagName);
                    if (thisClone) thisClone.remove();
                    return clone.textContent || '';
                }
            """)
            return (parent_text or "").strip()[:100]
        except:
            return ""
    
    async def _get_nearby_text(self, element: ElementHandle) -> List[str]:
        """Get text from nearby elements"""
        try:
            nearby = await element.evaluate("""
                el => {
                    const texts = [];
                    const parent = el.parentElement;
                    if (!parent) return texts;
                    
                    // Get previous sibling text
                    let prev = el.previousElementSibling;
                    if (prev && prev.textContent) {
                        texts.push(prev.textContent.trim());
                    }
                    
                    // Get next sibling text
                    let next = el.nextElementSibling;
                    if (next && next.textContent) {
                        texts.push(next.textContent.trim());
                    }
                    
                    // Get text from ancestors
                    let ancestor = parent.parentElement;
                    if (ancestor && ancestor.textContent) {
                        texts.push(ancestor.textContent.trim().substring(0, 50));
                    }
                    
                    return texts;
                }
            """)
            return [text for text in nearby if text][:5]
        except:
            return []

# ============================================================================
# SMART FORM INTERACTION
# ============================================================================

class SmartFormInteractor:
    """Intelligent form interaction with multiple strategies"""
    
    def __init__(self, page: Page):
        self.page = page
    
    async def interact_with_element(
        self, 
        selector: str, 
        value: str, 
        element_analysis: FormElementAnalysis
    ) -> bool:
        """Smart interaction based on element analysis"""
        try:
            logger.log_interaction("Interacting", selector, "started", {
                "type": element_analysis.element_type.value,
                "strategy": element_analysis.interaction_strategy
            })
            
            # Use Playwright locator for interactions
            locator = self.page.locator(selector).first
            
            # Wait for element to be ready
            try:
                await locator.wait_for(state="visible", timeout=5000)
            except:
                logger.log_warning(f"Element not visible: {selector}")
                # Try to interact anyway
            
            # Apply interaction strategy
            if element_analysis.element_type == ElementType.SELECT:
                return await self._handle_select(locator, value, element_analysis)
            elif element_analysis.element_type in [ElementType.TEXT_INPUT, ElementType.EMAIL_INPUT]:
                return await self._handle_text_input(locator, value, element_analysis)
            elif element_analysis.element_type == ElementType.TEXTAREA:
                return await self._handle_textarea(locator, value, element_analysis)
            elif element_analysis.element_type == ElementType.CHECKBOX:
                return await self._handle_checkbox(locator, value, element_analysis)
            elif element_analysis.element_type == ElementType.RADIO:
                return await self._handle_radio(locator, value, element_analysis)
            else:
                # Default text input handling
                return await self._handle_text_input(locator, value, element_analysis)
                
        except Exception as e:
            logger.log_error(f"Failed to interact with {selector}", e)
            return False
    
    async def _handle_text_input(
        self, 
        element: Locator, 
        value: str, 
        analysis: FormElementAnalysis
    ) -> bool:
        """Handle text input with smart strategies"""
        try:
            # Check if element needs to be enabled first
            if not await element.is_enabled():
                logger.log_ai_insight("Element is disabled, looking for enable trigger")
                # Look for nearby enable buttons/checkboxes
                parent = element.locator('..')
                enable_buttons = parent.locator('button, input[type="checkbox"]')
                if await enable_buttons.count() > 0:
                    await enable_buttons.first.click()
                    await asyncio.sleep(0.5)
            
            # Clear and fill
            await element.click()
            await element.clear()
            await element.fill(value)
            
            # Trigger change events
            await element.dispatch_event('change')
            await element.dispatch_event('input')
            
            # Verify
            actual_value = await element.input_value()
            success = value.lower() in actual_value.lower()
            
            logger.log_interaction("Text input", analysis.selector, 
                                 "success" if success else "failed", 
                                 {"expected": value, "actual": actual_value})
            return success
            
        except Exception as e:
            logger.log_error(f"Failed to handle text input", e)
            return False
    
    async def _handle_select(
        self, 
        element: Locator, 
        value: str, 
        analysis: FormElementAnalysis
    ) -> bool:
        """Handle select with fuzzy matching"""
        try:
            # Get all options
            options = await element.locator('option').all_text_contents()
            
            # Find best match
            best_match = None
            best_score = 0
            value_lower = value.lower()
            
            for i, option_text in enumerate(options):
                option_lower = option_text.lower()
                
                # Exact match
                if value_lower == option_lower:
                    best_match = option_text
                    break
                
                # Contains match
                if value_lower in option_lower or option_lower in value_lower:
                    score = len(value_lower) / len(option_lower)
                    if score > best_score:
                        best_score = score
                        best_match = option_text
            
            if best_match:
                await element.select_option(label=best_match)
                logger.log_interaction("Select", analysis.selector, "success", 
                                     {"selected": best_match})
                return True
            else:
                logger.log_interaction("Select", analysis.selector, "no match found", 
                                     {"options": options[:5]})
                return False
                
        except Exception as e:
            logger.log_error(f"Failed to handle select", e)
            return False
    
    async def _handle_textarea(
        self, 
        element: Locator, 
        value: str, 
        analysis: FormElementAnalysis
    ) -> bool:
        """Handle textarea with formatting preservation"""
        try:
            await element.click()
            await element.clear()
            
            # Use type for multi-line text to preserve formatting
            await element.type(value, delay=10)
            
            # Verify
            actual_value = await element.input_value()
            success = len(actual_value) > len(value) * 0.8  # 80% threshold
            
            logger.log_interaction("Textarea", analysis.selector, 
                                 "success" if success else "failed",
                                 {"length_expected": len(value), 
                                  "length_actual": len(actual_value)})
            return success
            
        except Exception as e:
            logger.log_error(f"Failed to handle textarea", e)
            return False
    
    async def _handle_checkbox(
        self, 
        element: Locator, 
        value: str, 
        analysis: FormElementAnalysis
    ) -> bool:
        """Handle checkbox based on value intent"""
        try:
            is_checked = await element.is_checked()
            should_check = value.lower() in ['true', 'yes', '1', 'check', 'checked', 'on']
            
            if should_check != is_checked:
                await element.click()
            
            logger.log_interaction("Checkbox", analysis.selector, "success",
                                 {"checked": should_check})
            return True
            
        except Exception as e:
            logger.log_error(f"Failed to handle checkbox", e)
            return False
    
    async def _handle_radio(
        self, 
        element: Locator, 
        value: str, 
        analysis: FormElementAnalysis
    ) -> bool:
        """Handle radio button selection"""
        try:
            # Find radio group
            name = await element.get_attribute('name')
            if name:
                # Find matching radio in group
                radios = self.page.locator(f'input[type="radio"][name="{name}"]')
                count = await radios.count()
                
                for i in range(count):
                    radio = radios.nth(i)
                    label_text = await self._get_radio_label(radio)
                    
                    if value.lower() in label_text.lower():
                        await radio.click()
                        logger.log_interaction("Radio", analysis.selector, "success",
                                             {"selected": label_text})
                        return True
            
            logger.log_interaction("Radio", analysis.selector, "no match")
            return False
            
        except Exception as e:
            logger.log_error(f"Failed to handle radio", e)
            return False
    
    async def _get_radio_label(self, radio: Locator) -> str:
        """Get label text for radio button"""
        try:
            # Check for associated label
            radio_id = await radio.get_attribute('id')
            if radio_id:
                label = self.page.locator(f'label[for="{radio_id}"]')
                if await label.count() > 0:
                    return await label.text_content()
            
            # Check next sibling
            parent = radio.locator('..')
            text = await parent.text_content()
            return text.strip() if text else ""
            
        except:
            return ""

# ============================================================================
# MAIN AGENTIC FORM FILLER
# ============================================================================

class AgenticAIFormFiller:
    """Main form filler with full AI reasoning"""
    
    def __init__(self, api_key: str):
        self.anthropic_client = anthropic.Anthropic(api_key=api_key)
        self.instructor_client = instructor.from_anthropic(self.anthropic_client)
        self.playwright = None
        self.browser = None
        self.page = None
        self.ai_analyzer = AIElementAnalyzer(self.anthropic_client)
        
        logger.log_success("Agentic AI Form Filler initialized")
    
    async def start(self):
        """Initialize browser"""
        try:
            logger.logger.info("Starting browser...")
            
            self.playwright = await async_playwright().start()
            self.browser = await self.playwright.chromium.launch(
                headless=False,
                args=['--no-sandbox']
            )
            
            context = await self.browser.new_context(
                viewport={'width': 1280, 'height': 720}
            )
            self.page = await context.new_page()
            
            logger.log_success("Browser started")
            
        except Exception as e:
            logger.log_error("Failed to start browser", e)
            raise
    
    async def stop(self):
        """Clean up"""
        try:
            if self.browser:
                await self.browser.close()
            if self.playwright:
                await self.playwright.stop()
            logger.logger.info("Browser closed")
        except Exception as e:
            logger.log_error("Error during cleanup", e)
    
    async def fill_form(self, url: str, form_data: Dict[str, str]) -> Dict[str, bool]:
        """Fill form using AI-driven approach"""
        try:
            logger.logger.info(f"Navigating to: {url}")
            logger.logger.info(f"Form data to fill: {list(form_data.keys())}")
            
            await self.page.goto(url, wait_until="domcontentloaded")
            await asyncio.sleep(2)
            
            # Extract all form elements
            extractor = SmartElementExtractor(self.page)
            element_contexts = []
            
            # Get all form inputs
            selectors = [
                'input:not([type="hidden"])',
                'textarea',
                'select',
                'button[type="submit"], button:has-text("submit"), input[type="submit"]'
            ]
            
            for selector in selectors:
                elements = await self.page.query_selector_all(selector)
                logger.logger.info(f"Found {len(elements)} {selector} elements")
                
                for element in elements:
                    element_data = await extractor.extract_element_data(element)
                    if element_data:
                        element_contexts.append(element_data)
            
            logger.log_success(f"Extracted {len(element_contexts)} form elements")
            
            # Get AI understanding of form
            form_understanding = await self.ai_analyzer.analyze_form_elements(element_contexts)
            
            # Match fields to elements with AI - pass the FORM DATA not element data!
            matching_decisions = await self.ai_analyzer.match_fields_to_elements(
                form_data, form_understanding  # Changed from 'data' to 'form_data'
            )
            
            # Build element analysis lookup
            element_analysis_map = {
                elem.selector: elem for elem in form_understanding.elements
            }
            
            # Execute interactions
            interactor = SmartFormInteractor(self.page)
            results = {}
            
            # Process in recommended order if available
            field_order = list(form_data.keys())
            if form_understanding.interaction_order:
                # Reorder based on AI recommendation
                ordered_fields = []
                for selector in form_understanding.interaction_order:
                    for decision in matching_decisions:
                        if decision.matched_selector == selector:
                            ordered_fields.append(decision.field_name)
                # Add any remaining fields
                for field in field_order:
                    if field not in ordered_fields:
                        ordered_fields.append(field)
                field_order = ordered_fields
            
            # Fill fields
            for field_name in field_order:
                # Find matching decision
                decision = next((d for d in matching_decisions if d.field_name == field_name), None)
                
                if decision and decision.matched_selector and decision.confidence > 0.3:
                    logger.log_ai_insight(
                        f"Filling '{field_name}' using selector '{decision.matched_selector}'",
                        {"confidence": decision.confidence, "reasoning": decision.reasoning}
                    )
                    
                    # Get element analysis
                    element_analysis = element_analysis_map.get(
                        decision.matched_selector,
                        FormElementAnalysis(
                            selector=decision.matched_selector,
                            element_type=ElementType.TEXT_INPUT,
                            field_purpose=FieldPurpose.CUSTOM_TEXT,
                            semantic_name=field_name,
                            confidence=0.5,
                            reasoning="Default analysis",
                            expected_content="text",
                            interaction_strategy="standard fill"
                        )
                    )
                    
                    # Apply value transformation if needed
                    value = form_data[field_name]
                    if decision.transformation_needed:
                        logger.log_ai_insight(f"Applying transformation: {decision.transformation_needed}")
                        # Could implement actual transformations here
                    
                    # Interact with element
                    success = await interactor.interact_with_element(
                        decision.matched_selector,
                        value,
                        element_analysis
                    )
                    
                    results[field_name] = success
                    
                    # Small delay between fields
                    await asyncio.sleep(0.2)
                else:
                    logger.logger.warning(f"No confident match for field: {field_name}")
                    results[field_name] = False
            
            # Log final results
            logger.session_data["final_results"] = results
            success_count = sum(results.values())
            total_count = len(results)
            
            logger.log_success(
                f"Form filling completed: {success_count}/{total_count} fields successful"
            )
            
            return results
            
        except Exception as e:
            logger.log_error("Critical error during form filling", e)
            return {field: False for field in form_data.keys()}

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def generate_test_card_data() -> Dict[str, str]:
    """Generate test Magic card data"""
    return {
        "name": "Temporal Dragon Lord",
        "cost": "4UUR",
        "type": "Legendary Creature",
        "subtype": "Elder Dragon",
        "rules": "Flying, Vigilance\n\nWhen Temporal Dragon Lord enters the battlefield, take an extra turn after this one.\n\n{2}{U}{U}: Return target nonland permanent to its owner's hand.",
        "power": "7",
        "toughness": "7",
        "rarity": "mythic rare"
    }

class CardGenerator:
    """Generate Magic cards using AI"""
    
    def __init__(self, instructor_client):
        self.client = instructor_client
    
    async def generate(self, concept: str) -> Dict[str, str]:
        """Generate a card from concept"""
        try:
            class Card(BaseModel):
                name: str
                cost: str
                type: str
                subtype: str = ""
                rules: str
                power: Optional[str] = None
                toughness: Optional[str] = None
                rarity: str = "rare"
            
            response = await asyncio.to_thread(
                self.client.chat.completions.create,
                model="claude-3-5-sonnet-20241022",
                response_model=Card,
                messages=[{
                    "role": "user",
                    "content": f"Create a Magic: The Gathering card based on: {concept}"
                }],
                max_tokens=400
            )
            
            # Convert to dict
            card_dict = {
                "name": response.name,
                "cost": response.cost,
                "type": response.type,
                "subtype": response.subtype,
                "rules": response.rules,
                "rarity": response.rarity
            }
            
            if response.power:
                card_dict["power"] = response.power
            if response.toughness:
                card_dict["toughness"] = response.toughness
                
            logger.log_success(f"Generated card: {response.name}")
            return card_dict
            
        except Exception as e:
            logger.log_error("Failed to generate card", e)
            return generate_test_card_data()

# ============================================================================
# EXAMPLE USAGE
# ============================================================================

async def main():
    """Example usage with Magic card form"""
    filler = None
    try:
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if not api_key:
            print("❌ Please set ANTHROPIC_API_KEY environment variable")
            return
        
        # Initialize filler
        filler = AgenticAIFormFiller(api_key)
        
        await filler.start()
        
        # Generate or use test card data
        print("🎴 Generating Magic card data...")
        generator = CardGenerator(filler.instructor_client)
        card_data = await generator.generate("A dragon that controls time and space")
        
        print(f"\n📝 Card Data:")
        for field, value in card_data.items():
            print(f"  - {field}: {value}")
        
        # Fill form with card data
        print(f"\n🔄 Starting form filling process...")
        results = await filler.fill_form(
            "https://mtgcardsmith.com/mtg-card-maker/edit",
            card_data
        )
            
        
        # Display results
        print("\n" + "="*60)
        print("FORM FILLING RESULTS")
        print("="*60)
        
        for field, success in results.items():
            status = "✅" if success else "❌"
            print(f"{status} {field}")
        
        success_rate = sum(results.values()) / len(results) * 100 if results else 0
        print(f"\nSuccess Rate: {success_rate:.1f}%")
        
        # Show AI insights
        print("\n" + "="*60)
        print("AI INSIGHTS")
        print("="*60)
        
        for insight in logger.session_data["ai_insights"][-10:]:  # Show last 10
            print(f"🤖 {insight['insight']}")
        
        # Get session summary
        summary = logger.get_session_summary()
        print(f"\nSession Summary:")
        print(f"- Duration: {summary['duration']:.1f}s")
        print(f"- AI Insights: {summary['ai_insights']}")
        print(f"- Interactions: {summary['interactions']}")
        print(f"- Errors: {summary['errors']}")
        
        # Keep browser open
        print("\n✋ Browser will remain open for 30 seconds for inspection...")
        await asyncio.sleep(30)
            
    except KeyboardInterrupt:
        print("\n⚠️  Interrupted by user")
    except Exception as e:
        logger.log_error("Critical error in main", e)
        print(f"\n❌ Critical error: {e}")
        print("Check the logs above for details")
    finally:
        if filler:
            await filler.stop()
        print("\n👋 Form filler session ended")

if __name__ == "__main__":
    print("🚀 Starting AI-Driven Agentic Form Filler")
    print("="*60)
    asyncio.run(main())

Task exception was never retrieved
future: <Task finished name='Task-1239' coro=<Connection.run.<locals>.init() done, defined at /home/joey/.conda/envs/ml_shit/lib/python3.11/site-packages/playwright/_impl/_connection.py:276> exception=Exception('Connection.init: Connection closed while reading from the driver')>
Traceback (most recent call last):
  File "/home/joey/.conda/envs/ml_shit/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/home/joey/.conda/envs/ml_shit/lib/python3.11/site-packages/playwright/_impl/_connection.py", line 277, in init
    self.playwright_future.set_result(await self._root_object.initialize())
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/joey/.conda/envs/ml_shit/lib/python3.11/site-packages/playwright/_impl/_connection.py", line 219, in initialize
    await self._channel.send(
  File "/home/joey/.conda/envs/ml_shit/lib/python3.11/site-packages/play

🚀 Starting AI-Driven Agentic Form Filler


13:04:45 | INFO     | log_success          | ✅ Browser started
13:04:45 | INFO     | log_success          | ✅ Browser started
13:04:45 | INFO     | log_success          | ✅ Browser started
13:04:45 | INFO     | log_success          | ✅ Browser started
13:04:45 | INFO     | log_success          | ✅ Browser started


🎴 Generating Magic card data...


13:04:49 | INFO     | log_success          | ✅ Generated card: Chronospace Dragon
13:04:49 | INFO     | log_success          | ✅ Generated card: Chronospace Dragon
13:04:49 | INFO     | log_success          | ✅ Generated card: Chronospace Dragon
13:04:49 | INFO     | log_success          | ✅ Generated card: Chronospace Dragon
13:04:49 | INFO     | log_success          | ✅ Generated card: Chronospace Dragon
13:04:49 | INFO     | fill_form            | Navigating to: https://mtgcardsmith.com/mtg-card-maker/edit
13:04:49 | INFO     | fill_form            | Navigating to: https://mtgcardsmith.com/mtg-card-maker/edit
13:04:49 | INFO     | fill_form            | Navigating to: https://mtgcardsmith.com/mtg-card-maker/edit
13:04:49 | INFO     | fill_form            | Navigating to: https://mtgcardsmith.com/mtg-card-maker/edit
13:04:49 | INFO     | fill_form            | Navigating to: https://mtgcardsmith.com/mtg-card-maker/edit
13:04:49 | INFO     | fill_form            | Form data to fill: [


📝 Card Data:
  - name: Chronospace Dragon
  - cost: 4UURR
  - type: Creature
  - subtype: Elder Dragon
  - rules: Flying
When Chronospace Dragon enters the battlefield, take an extra turn after this one.
{2}{U}{R}: Exile target permanent. Return it to the battlefield under its owner's control at the beginning of their next upkeep.
At the beginning of your upkeep, scry 2.
  - rarity: mythic rare
  - power: 6
  - toughness: 6

🔄 Starting form filling process...


13:04:58 | INFO     | fill_form            | Found 53 input:not([type="hidden"]) elements
13:04:58 | INFO     | fill_form            | Found 53 input:not([type="hidden"]) elements
13:04:58 | INFO     | fill_form            | Found 53 input:not([type="hidden"]) elements
13:04:58 | INFO     | fill_form            | Found 53 input:not([type="hidden"]) elements
13:04:58 | INFO     | fill_form            | Found 53 input:not([type="hidden"]) elements
13:05:49 | INFO     | fill_form            | Found 1 textarea elements
13:05:49 | INFO     | fill_form            | Found 1 textarea elements
13:05:49 | INFO     | fill_form            | Found 1 textarea elements
13:05:49 | INFO     | fill_form            | Found 1 textarea elements
13:05:49 | INFO     | fill_form            | Found 1 textarea elements
13:05:50 | INFO     | fill_form            | Found 12 select elements
13:05:50 | INFO     | fill_form            | Found 12 select elements
13:05:50 | INFO     | fill_form            | Found 12 s


FORM FILLING RESULTS
✅ name
✅ cost
❌ type
✅ subtype
❌ rarity
❌ power
❌ toughness
❌ rules

Success Rate: 37.5%

AI INSIGHTS
🤖 Matched 'rarity' to '#s2id_autogen2' (confidence: 0.90)
🤖 Matched 'power' to '#power' (confidence: 1.00)
🤖 Matched 'toughness' to '#toughness' (confidence: 1.00)
🤖 Filling 'name' using selector '#cardName'
🤖 Filling 'cost' using selector '#custom_mana'
🤖 Filling 'type' using selector '#s2id_autogen3'
🤖 Filling 'subtype' using selector '#subtype'
🤖 Filling 'rarity' using selector '#s2id_autogen2'
🤖 Filling 'power' using selector '#power'
🤖 Filling 'toughness' using selector '#toughness'

Session Summary:
- Duration: 130.6s
- AI Insights: 17
- Interactions: 12
- Errors: 2

✋ Browser will remain open for 30 seconds for inspection...


13:07:26 | INFO     | stop                 | Browser closed
13:07:26 | INFO     | stop                 | Browser closed
13:07:26 | INFO     | stop                 | Browser closed
13:07:26 | INFO     | stop                 | Browser closed
13:07:26 | INFO     | stop                 | Browser closed



👋 Form filler session ended
